In [57]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'

# data path_mi
path_csi = 'D:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'D:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'D:\\Data\\Wi-Fi_meta\\'
path_sc = 'D:\\Data\\Wi-Fi_info\\'
path_save = 'D:\\Data\\Wi-Fi_mit_image\\'

# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)


In [59]:
file = df_info.id.values[0]
file

'S017_1_2_11'

In [60]:
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)

In [78]:
s1 = data1.shape[0]
a = np.arange(500)
idx = (a * s1/500).astype('int')
data11 = data1[idx]
data11.shape

(500, 30, 2, 3)

In [5]:
# make data generator
def gen_csi(df_info,dict_avg,id_num,len_num):
    for file in np.unique(df_info.id.values):
        
        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]
        
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
            
        # subtract avg
        read_avg = dict_avg[id_key][0]
        nonzero_avg = read_avg[np.nonzero(read_avg)].reshape([-1,30,2,3])
        arr_avg = np.mean(nonzero_avg,axis=0)
        data1_diff = data1 - arr_avg #np.diff(data1,axis=0)
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # subcarrier info
        #data1_sc_df = pd.read_csv(path_sc + file + '_df_sc.csv')
        #data1_time = np.diff(data1_sc_df['timestamp_low'])
        #data1_time_pad = np.pad(data1_time,((1,pad_len)),'constant',constant_values=0)
        
        
        yield(data1_pad ,data1_y,id_key)#,data1_time_pad)

In [6]:
gen = gen_csi(df_info,dict_avg,no_classes,csi_time)
target_sig,target_lab,target_id,target_dt = next(gen)

In [7]:
# 3D scan
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 30

th_range,si_range = (30,30)
sig_mat = np.zeros([csi_time,2*th_range,2*si_range])

In [41]:
# 3D Reconstruction func
from numba import vectorize
@vectorize(['complex64(complex64,float32,float32,int32,int32,float32,float32,float32,int32)'], target='cpu')
def Recon3d(sig,theta,sigma,m,n,lam,d,r,c):
    #above_eq1 = 1j * (2*math.pi) * k * r * t / c
    #above_eq1 = 1j * (2*math.pi) * c * dt * 0.000001 / lam
    above_eq1 = 1j * (2*math.pi) * (160 + 160 + 164) * 0.01 / lam
    above_eq2 = 1j * (2*math.pi/lam) * math.sin(theta) * ((n+1)*d*math.cos(sigma) + (m+1)*d*math.sin(sigma))
    eq_res = sig* cmath.exp(above_eq1) * cmath.exp(above_eq2)
    #eq_res = cmath.exp(above_eq1) * cmath.exp(above_eq2)
    return eq_res #math.atan2(eq_res.imag,eq_res.real)


In [54]:
def Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n):
    #vectorize arrays
    subc_array = np.ones([csi_time,max_subc,m,n])
    m_array = np.ones([csi_time,max_subc,m,n])
    n_array = np.ones([csi_time,max_subc,m,n])
    for i in range(max_subc):
        subc_array[:,i,:,:] = i
    for i in range(m):
        m_array[:,:,i,:] = i
    for i in range(n):
        n_array[:,:,:,i] = i
    subc_array = subc_array.flatten().astype(np.int32)
    m_array = m_array.flatten().astype(np.int32)
    n_array = n_array.flatten().astype(np.int32)
    lam_array = np.array([c*0.000001/bw_list[str(e)][ch] for e in subc_array],dtype=np.float32)
    
    target_array = target_sig.flatten().astype(np.complex64)
    
    #3d recon
    sum_eq = np.zeros(csi_time*max_subc*m*n*th_range,dtype=np.complex_)
    sum_eq = Recon3d(target_array,theta,sigma,m_array,n_array,lam_array,d,r,c)

    sum_reshape = np.sum(sum_eq.reshape([-1,max_subc,m,n]),axis=(1,2,3))
    return(np.abs(sum_reshape))

In [55]:
import time
t1 = time.time()

for idx_th,i in enumerate(range(-th_range,th_range)):
    for idx_si,j in enumerate(range(-si_range,si_range)):
        theta = i * (np.radians(30)/2) / th_range
        sigma = j * (np.radians(30)/2) / si_range
print(time.time()-t1)

0.009995698928833008


In [56]:
res = Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n)

In [20]:
th_array = (np.radians(30)/2)/th_range * np.arange(-th_range,th_range)
si_array = (np.radians(30)/2)/si_range * np.arange(-si_range,si_range)

In [34]:
len(th_array)

60

In [48]:
np.concatenate([th_array,th_array])

array([-0.26179939, -0.25307274, -0.2443461 , -0.23561945, -0.2268928 ,
       -0.21816616, -0.20943951, -0.20071286, -0.19198622, -0.18325957,
       -0.17453293, -0.16580628, -0.15707963, -0.14835299, -0.13962634,
       -0.13089969, -0.12217305, -0.1134464 , -0.10471976, -0.09599311,
       -0.08726646, -0.07853982, -0.06981317, -0.06108652, -0.05235988,
       -0.04363323, -0.03490659, -0.02617994, -0.01745329, -0.00872665,
        0.        ,  0.00872665,  0.01745329,  0.02617994,  0.03490659,
        0.04363323,  0.05235988,  0.06108652,  0.06981317,  0.07853982,
        0.08726646,  0.09599311,  0.10471976,  0.1134464 ,  0.12217305,
        0.13089969,  0.13962634,  0.14835299,  0.15707963,  0.16580628,
        0.17453293,  0.18325957,  0.19198622,  0.20071286,  0.20943951,
        0.21816616,  0.2268928 ,  0.23561945,  0.2443461 ,  0.25307274,
       -0.26179939, -0.25307274, -0.2443461 , -0.23561945, -0.2268928 ,
       -0.21816616, -0.20943951, -0.20071286, -0.19198622, -0.18

In [50]:
len(np.tile(th_array,2))

120